In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display
import requests as re
from io import BytesIO
from os import path, makedirs


from core.downloads.geosampa import get_capabilities, get_features

# GT Saúde

O Grupo de Trabalho de Saúde definiu os indicadores sem a utilização dos formulários, em reuniões posteriores. Por isso, os indicadores serão descritos diretamente neste notebook para documentação.

A base inicial para os indicadores é o capítulo 7, Eficiência nas macrorregiões de saúde no Sistema Único de Saúde: uma abordagem comparativa – 2008-2017, do livro [SUS: avaliação da eficiência do gasto público em saúde](https://repositorio.ipea.gov.br/handle/11058/12029). Este capítulo busca associar os custos e infraestrutura relacionada ao SUS e o impacto na saúde da população de cada macrorregião, utilizando as seguintes variáveis:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Despesas per capita em atenção básica e vigilância;
- Despesas per capita em atenção especializada e Suporte profilático terapêutico;
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Equipamentos respiradores e ventiladores SUS/hab. (x100.000);
- População (em mil hab.);
- Área (km2);
- Densidade demográfica (hab./km²);
- Idosos %;
- PIB per capita;
- Usuários de planos de saúde %.


A partir dessas referência, o GT avaliou a disponibilidade de dados semelhantes a nível intramunicipal e complementou com informações de produção dos serviços de saúde, chegando na seguinte lista de variáveis, sempre que possível agregando por Subprefeitura:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Leitos de UTI/hab. (x100.000);
- Despesas liquidadas em atenção básica e vigilância;
- Despesas liquidadas em atenção especializada e Suporte profilático terapêutico;
- Consultas na Atenção Básica;
- Consultas na Atenção Especializada;
- Consultas na Urgência/Emergência;
- Número de Unidades Básicas de Saúde;
- Número de Ambulatórios especializados;
- Número de Unidades de Saúde Mental;
- Número de Unidades de Urgência e Emergência;
- Número de Hospitais;
- Número de Unidades DST/AIDS;
- Número de demais unidades;

A partir dessa definição, vamos obter as variáveis.

# Extração das variáveis e transformação inicial

## Número de unidades

Estabelecimentos de saúde municipais, estaduais, federais e privados do município estão disponíveis no GeoSampa, em diferentes camadas. Primeiro, vamos avaliar quais são essas camadas e, depois, baixá-las e agrupá-las da maneira decidida pelo GT.

In [2]:
equipamento_saude_typenames = get_capabilities('equipamento_saude')
equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_abrangencia_ubs',
  'title': 'Abrangência das Unidades Básicas de Saúde',
  'abstract': 'Áreas de abrangência das UBS.'},
 {'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_coordenadoria_regional',
  'title': 'Coordenadoria Regional de Saúde',
  'abstract': 'Limites administrativos da Coordenadoria Regional de Saúde.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que real

Primeiro, removemos todas as camadas que não representam os pontos das unidades de saúde. Além disso, a camada de vigilância não retorna nenhuma unidade, então vamos removê-la também.

In [3]:
del equipamento_saude_typenames[0]
del equipamento_saude_typenames[1]
del equipamento_saude_typenames[4]
del equipamento_saude_typenames[8]
del equipamento_saude_typenames[7]

equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que realizam procedimentos de média e alta complexidade.'},
 {'name': 'geoportal:equipamento_saude_outros',
  'title': 'Saúde - Outros',
  'abstract': 'Estabelecimentos e serviços especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo.'},
 {'name': 'geoportal:equipamento_saude_saude_mental

In [4]:
gdfs_equip = []

for t in equipamento_saude_typenames:
    gdfs_equip.append(get_features(t.get('name')))

In [5]:
for gdf in gdfs_equip:
    display(gdf.crs)
    display(gdf.head(2))

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,cd_identificador_classe_equipamento,nm_classe_equipamento,tx_classe_equipamento,cd_identificador_esfera_administrativa_equipamento,nm_esfera_administrativa_equipamento,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,963,AMA 12H BORACEA - DR. LUIZ BACCALA,"R. DR. RIBEIRO DE ALMEIDA, 14",BARRA FUNDA,01137-020,None,4210-5396 / 3392-1944,None,POINT (331397.001 7397422.322)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,935,AMA 12H COMPLEXO PRATES,"R. PRATES, 1101",BOM RETIRO,01121-000,None,3229-8192,None,POINT (333135.515 7397559.623)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,cd_identificador_classe_equipamento,nm_classe_equipamento,tx_classe_equipamento,cd_identificador_esfera_administrativa_equipamento,nm_esfera_administrativa_equipamento,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_hospital.120093,120093,6,SAUDE,0,SEM REDE,Sem descrição,32,Hospital Especializado,É o estabelecimento de saúde destinado a prest...,7,HOSPITAL,REFERE-SE AOS ESTABELECIMENTOS DE SAÚDE COM CA...,2,Estadual,Administração estadual vinculada à Secretaria ...,959,AMB ESPEC DR. GERALDO PAULO BOURROL (CONSOLAÇÃO),"R. MARTINS FONTES, 208",CENTRO,01050-000,None,2141-6200,None,POINT (332182.763 7394765.185)
1,equipamento_saude_hospital.120108,120108,6,SAUDE,0,SEM REDE,Sem descrição,32,Hospital Especializado,É o estabelecimento de saúde destinado a prest...,7,HOSPITAL,REFERE-SE AOS ESTABELECIMENTOS DE SAÚDE COM CA...,2,Estadual,Administração estadual vinculada à Secretaria ...,366,AME DR. LUIZ ROBERTO BARRADAS BARATA,"AV. ALMIRANTE DELAMARE, 1534",CIDADE NOVA HELIOPOLIS,04230-000,None,2065-1550,None,POINT (337785.173 7388015.086)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

...

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,cd_identificador_classe_equipamento,nm_classe_equipamento,tx_classe_equipamento,cd_identificador_esfera_administrativa_equipamento,nm_esfera_administrativa_equipamento,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ubs_posto_centro.120031,120031,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",5,UBS/POSTO DE SAÚDE/CENTRO DE SAÚDE,O PRIMEIRO CONTATO DO PACIENTE COM O SUS.,1,Municipal,Administração municipal,580,AMA/UBS ÁGUA RASA - DR. MARCOS ANDRADE CORSATO,"R. SERRA DE JAIRE, 1480",ÁGUA RASA,03175-001,None,2605-5307,None,POINT (338714.702 7394301.385)
1,equipamento_saude_ubs_posto_centro.120032,120032,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",5,UBS/POSTO DE SAÚDE/CENTRO DE SAÚDE,O PRIMEIRO CONTATO DO PACIENTE COM O SUS.,1,Municipal,Administração municipal,429,AMA/UBS AGUIA DE HAIA,"R. TANTAS PALAVRAS, 59",COHAB ÁGUIA DE HAIA,08223-510,None,3756-3264,None,POINT (350002.697 7397919.976)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,cd_identificador_classe_equipamento,nm_classe_equipamento,tx_classe_equipamento,cd_identificador_esfera_administrativa_equipamento,nm_esfera_administrativa_equipamento,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_unidades_dst-aids.120363,120363,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,8,UNIDADES DST/AIDS,"PREVENÇÃO, DIAGNÓSTICO E TRATAMENTO DE PESSOAS...",1,Municipal,Administração municipal,224,CTA IST/AIDS CIDADE TIRADENTES,"R. MILAGRE DOS PEIXES, 357",CONJUNTO HABITACIONA,08474-120,None,2282-7055 / 2964-0784,None,POINT (356240.559 7391275.466)
1,equipamento_saude_unidades_dst-aids.120364,120364,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,8,UNIDADES DST/AIDS,"PREVENÇÃO, DIAGNÓSTICO E TRATAMENTO DE PESSOAS...",1,Municipal,Administração municipal,319,CTA IST/AIDS GUAIANASES,"R. CENTRALINA, 168",GUAIANASES,08410-000,None,2554-5312,None,POINT (355449.169 7395443.615)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,cd_identificador_classe_equipamento,nm_classe_equipamento,tx_classe_equipamento,cd_identificador_esfera_administrativa_equipamento,nm_esfera_administrativa_equipamento,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_urgencia_emergencia.120007,120007,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",6,URGÊNCIA/EMERGÊNCIA,ATENDE PACIENTE QUE REQUER CUIDADO IMEDIATO E ...,1,Municipal,Administração municipal,78,AMA 24H CAPÃO REDONDO,"AV. COMENDADOR SANTANA, 774",JD BOA ESPERANÇA,05866-000,None,5872-9917 / 5872-9919,None,POINT (319131.694 7380616.296)
1,equipamento_saude_urgencia_emergencia.120008,120008,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",6,URGÊNCIA/EMERGÊNCIA,ATENDE PACIENTE QUE REQUER CUIDADO IMEDIATO E ...,1,Municipal,Administração municipal,646,AMA 24H ENGENHEIRO GOULART - DR. JOSÉ PIRES,"R. AUGUSTO CORREIA LEITE, 538",ENG. GOULART,03726-100,None,2038-1626 / 2622-0366,None,POINT (345081.544 7400275.725)


In [6]:
gdf_equip = pd.concat(gdfs_equip)
gdf_equip = gpd.GeoDataFrame(gdf_equip)
gdf_equip = gdf_equip.set_geometry(gdfs_equip[0].geometry.name)
gdf_equip = gdf_equip.set_crs(gdfs_equip[0].crs)
gdf_equip

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,cd_identificador_classe_equipamento,nm_classe_equipamento,tx_classe_equipamento,cd_identificador_esfera_administrativa_equipamento,nm_esfera_administrativa_equipamento,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,963,AMA 12H BORACEA - DR. LUIZ BACCALA,"R. DR. RIBEIRO DE ALMEIDA, 14",BARRA FUNDA,01137-020,None,4210-5396 / 3392-1944,None,POINT (331397.001 7397422.322)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,935,AMA 12H COMPLEXO PRATES,"R. PRATES, 1101",BOM RETIRO,01121-000,None,3229-8192,None,POINT (333135.515 7397559.623)
2,equipamento_saude_ambulatorios_especializados....,120003,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,17,AMA 12H JARDIM PERI PERI,"R. JOÃO GUERRA, 247",JD PERI PERI,05535-100,None,3744-3021 / 3744-3082,None,POINT (323248.398 7390472.108)
3,equipamento_saude_ambulatorios_especializados....,120004,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,546,AMA 12H JARDIM SÃO LUIZ,"R. LUIZ ANTONIO VERNEY, 98",JD SÃO LUIZ,05846-380,None,5511-1153 / 5816-5539,None,POINT (322162.36 7382246.957)
4,equipamento_saude_ambulatorios_especializados....,120005,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",1,AMBULATÓRIOS ESPECIALIZADOS,ATENDE PACIENTES ENCAMINHADOS POR OUTROS ESTA...,1,Municipal,Administração municipal,658,AMA 12H MAURICE PATE,"R. FREI GERMANO, 50",PENHA,03604-050,None,2097-5802,None,POINT (341684.865 7397317.38)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,equipamento_saude_urgencia_emergencia.121025,121025,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,6,URGÊNCIA/EMERGÊNCIA,ATENDE PACIENTE QUE REQUER CUIDADO IMEDIATO E ...,1,Municipal,Administração municipal,543,UPA VERA CRUZ,"AV. DOS FUNCIONARIOS PUBLICOS, 379",JARDIM VERA CRUZ,04962-000,None,5898-5900,None,POINT (318482.017 7373748.846)
49,equipamento_saude_urgencia_emergencia.121026,121026,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,6,URGÊNCIA/EMERGÊNCIA,ATENDE PACIENTE QUE REQUER CUIDADO IMEDIATO E ...,1,Municipal,Administração municipal,952,UPA VERGUEIRO,"R. VERGUEIRO, 613",LIBERDADE,01504-001,None,4040-1067,None,POINT (332661.412 7392701.246)
50,equipamento_saude_urgencia_emergencia.121027,121027,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,6,URGÊNCIA/EMERGÊNCIA,ATENDE PACIENTE QUE REQUER CUIDADO IMEDIATO E ...,1,Municipal,Administração municipal,1008,UPA VILA MARIA - DR JOSE MAURO DEL ROIO CORREA,"PRAÇA ENGENHEIRO HUGO BRANDI, 15",VILA MARIA BAIXA,02168-050,None

## Consultas médicas

Os dados de consultas médicas serão obtidos do boletim "Saúde em dados", disponibilizado anualmente pela CEINFO, da Secretaria Municipal de Saúde. Existe uma versão do boletim em excel, regionalizado por subprefeituras. Primeiro, vamos confirmar as planilhas disponíveis no arquivo.

In [7]:
url = 'https://capital.sp.gov.br/documents/d/saude/tabelas_ceinfo_dados_sub_2024_v3_rev09012025'

response = re.get(url)
response.raise_for_status()
xlsx_file = BytesIO(response.content)
excel = pd.ExcelFile(xlsx_file)

excel.sheet_names

['População',
 'Nascidos Vivos',
 'Grupos de Robson',
 'Mortalidade',
 'Neoplasia',
 'Doenças Notificação Compulsória',
 'Agravos',
 'COVID-19',
 'Estado Nutricional',
 'Cobertura Assist SIGA-Saúde',
 'Consultas Medicas_Odontológicas',
 'Consultas Médicas Especializada',
 'Cons Prof Nivel Superior SMS',
 'Cons Prof Nivel Superior SES',
 'Assistência Hospitalar',
 'Apoio Diagnóstico',
 'Rede Física',
 'Leitos']

A tabela desejada é a `Consultas Medicas_Odontológicas`, então vamos abri-la como um dataframe.

In [8]:
sheet_name = 'Consultas Medicas_Odontológicas'
header = [0, 1]
skiprows = 5
df_consultas = pd.read_excel(xlsx_file,
                   sheet_name=sheet_name,
                   skiprows=skiprows,
                   header=header,
                   thousands='.')
df_consultas

Unnamed: 0_level_0                                      Subprefeitura  ... Total de Consultas Médicas Primeira Consulta Odontológica \nSMS
   Unnamed: 0_level_1                                 Unnamed: 1_level_1  ...                      Total                  Unnamed: 14_level_1
0                 NaN                          Aricanduva/Formosa/Carrão  ...                   400815.0                               5780.0
1                 NaN                                            Butantã  ...                  1020117.0                              12899.0
2                 NaN                                        Campo Limpo  ...                  1868686.0                              26734.0
3                 NaN                                  Capela do Socorro  ...                  1620328.0                              21709.0
4                 NaN                            Casa Verde/Cachoeirinha  ...                   973027.0                              10356.0
5                 NaN                                      Cidade Ademar  ...                  1107221.0                              16236.0
6                 NaN                                  Cidade Tiradentes  ...                   961839.0                              12190.0
7                 NaN                                 Ermelino Matarazzo  ...                   796660.0                               8022.0
8                 NaN                              Freguesia/Brasilândia  ...                  1264585.0                              17043.0
9                 NaN                                         Guaianases  ...                   846015.0                              11325.0
10                NaN                                           Ipiranga  ...                  1485620.0                              11929.0
11                NaN                                     Itaim Paulista  ...                   783492.0                              13679.0
12                NaN                                           Itaquera  ...                  1503418.0                              15336.0
13                NaN                                          Jabaquara  ...                   786335.0                               6390.0
14                NaN                                    Jaçanã/Tremembé  ...                   668650.0                              10688.0
15                NaN                                               Lapa  ...                   561511.0                               6517.0
16                NaN                                        M'Boi Mirim  ...                  2311911.0                              24665.0
17                NaN                                              Mooca  ...                  1506790.0                               6866.0
18                NaN                                        Parelheiros  ...                   628757.0                               9110.0
19                NaN                                              Penha  ...                  1215528.0                              12601.0
20                NaN                                              Perus  ...                   551643.0                               3364.0
21                NaN                                          Pinheiros  ...                  1792662.0                               2320.0
22                NaN                                   Pirituba/Jaraguá  ...                  1451442.0                              10422.0
23                NaN                                   Santana/Tucuruvi  ...                   795810.0                               4733.0
24                NaN                                        Santo Amaro  ...                   901752.0                               3999.0
25                NaN                                         São Mateus  ...                  1355396.0                              12035.0
26                NaN      

Como a tabela original vem em um formato amigável para humanos, precisamos ajustá-la para melhorar a sua utilização computacional. A primeira coisa é filtrar linhas e colunas carregadas por engano, por conter informações de fonte ou caracteres não imprimíveis.

In [9]:
df_consultas = df_consultas.iloc[:32, 1:]

df_consultas

Subprefeitura Consulta Médica\nna Atenção Básica(2)                ... Total de Consultas Médicas            Primeira Consulta Odontológica \nSMS
           Unnamed: 1_level_1                     Não Urgência\nSMS Urgência\nSMS  ...               Gestão \nSES      Total                  Unnamed: 14_level_1
0   Aricanduva/Formosa/Carrão                              188894.0       93846.0  ...                          -   400815.0                               5780.0
1                     Butantã                              443917.0      212283.0  ...                     181099  1020117.0                              12899.0
2                 Campo Limpo                              837251.0      652006.0  ...                      10531  1868686.0                              26734.0
3           Capela do Socorro                              554063.0      283894.0  ...                     181105  1620328.0                              21709.0
4     Casa Verde/Cachoeirinha                              327363.0      258095.0  ...                     178080   973027.0                              10356.0
5               Cidade Ademar                              645314.0      188878.0  ...                          -  1107221.0                              16236.0
6           Cidade Tiradentes                              325739.0       34662.0  ...                          -   961839.0                              12190.0
7          Ermelino Matarazzo                              258367.0      162909.0  ...                          -   796660.0                               8022.0
8       Freguesia/Brasilândia                              476514.0      120546.0  ...                      88974  1264585.0                              17043.0
9                  Guaianases                              358419.0       50300.0  ...                     127651   846015.0                              11325.0
10                   Ipiranga                              436257.0      253211.0  ...                     430586  1485620.0                              11929.0
11             Itaim Paulista                              506647.0        5370.0  ...                      58330   783492.0                              13679.0
12                   Itaquera                              616893.0      166986.0  ...                     246010  1503418.0                              15336.0
13                  Jabaquara                              202175.0      106137.0  ...                         87   786335.0                               6390.0
14            Jaçanã/Tremembé                              333609.0       68684.0  ...                          -   668650.0                              10688.0
15                       Lapa                              185920.0      191187.0  ...                      35962   561511.0                               6517.0
16                M'Boi Mirim                              927985.0      517440.0  ...                          -  2311911.0                              24665.0
17                      Mooca                              237620.0      118979.0  ...                     224023  1506790.0                               6866.0
18                Parelheiros                              165641.0       67455.0  ...                          -   628757.0                               9110.0
19                      Penha                              395582.0      564046.0  ...                          -  1215528.0                              12601.0
20                      Perus                              174019.0      135355.0  ...                          -   551643.0                               3364.0
21                  Pinheiros                               98130.0           3.0  ...                    1485997  1792662.0                               2320.0
22           Pirituba/Jaraguá                              499630.0      172618.0  ...                     202488  1451442.0    

Depois, precisamos remover as colunas de totais, que resultariam em duplicação dos valores caso fossem mantidas.

In [10]:
cols_total_level0 = df_consultas.columns.get_level_values(0).str.contains('total', case=False)
cols_total_level1 = df_consultas.columns.get_level_values(1).str.contains('total', case=False)

df_consultas = df_consultas.loc[:, ~cols_total_level0 & ~cols_total_level1]

df_consultas

Subprefeitura Consulta Médica\nna Atenção Básica(2)  ... Consulta Médica/Atendimento\nem Urgência/Emergência(3) Primeira Consulta Odontológica \nSMS
           Unnamed: 1_level_1                     Não Urgência\nSMS  ...                                           Gestão \nSES                  Unnamed: 14_level_1
0   Aricanduva/Formosa/Carrão                              188894.0  ...                                                  -                                   5780.0
1                     Butantã                              443917.0  ...                                              83033                                  12899.0
2                 Campo Limpo                              837251.0  ...                                                  4                                  26734.0
3           Capela do Socorro                              554063.0  ...                                              72301                                  21709.0
4     Casa Verde/Cachoeirinha                              327363.0  ...                                             116491                                  10356.0
5               Cidade Ademar                              645314.0  ...                                                  -                                  16236.0
6           Cidade Tiradentes                              325739.0  ...                                                  -                                  12190.0
7          Ermelino Matarazzo                              258367.0  ...                                                  -                                   8022.0
8       Freguesia/Brasilândia                              476514.0  ...                                              65486                                  17043.0
9                  Guaianases                              358419.0  ...                                             124415                                  11325.0
10                   Ipiranga                              436257.0  ...                                             175665                                  11929.0
11             Itaim Paulista                              506647.0  ...                                              31054                                  13679.0
12                   Itaquera                              616893.0  ...                                              31104                                  15336.0
13                  Jabaquara                              202175.0  ...                                                  -                                   6390.0
14            Jaçanã/Tremembé                              333609.0  ...                                                  -                                  10688.0
15                       Lapa                              185920.0  ...                                                  -                                   6517.0
16                M'Boi Mirim                              927985.0  ...                                                  -                                  24665.0
17                      Mooca                              237620.0  ...                                              98139                                   6866.0
18                Parelheiros                              165641.0  ...                                                  -                                   9110.0
19                      Penha                              395582.0  ...                                                  -                                  12601.0
20                      Perus                              174019.0  ...                                                  -                                   3364.0
21                  Pinheiros                               98130.0  ...                                              75159                                   2320.0
22           Pirituba/Jaraguá                           

Agora, vamos transformar os níveis das colunas em novas colunas de valores (despivotar a tabela).

In [11]:
id_col = df_consultas.columns[0]
df_consultas_ajustado = pd.melt(df_consultas, id_vars=[id_col])
df_consultas_ajustado

,"(Subprefeitura, Unnamed: 1_level_1)",variable_0,variable_1,value
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
219,Sapopemba,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,9873.0
220,Sé,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,27847.0
221,Vila Maria/Vila Guilherme,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,16263.0
222,Vila Mariana,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,3015.0


Agora, renomeamos as colunas.

In [12]:
df_consultas_ajustado.columns = ['Subprefeitura', 'Categoria', 'Subcategoria', 'Qtd_Consultas']
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
219,Sapopemba,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,9873.0
220,Sé,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,27847.0
221,Vila Maria/Vila Guilherme,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,16263.0
222,Vila Mariana,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,3015.0


Em seguida, removemos as linhas de consultas odontológicas.

In [13]:
df_consultas_ajustado = df_consultas_ajustado[~df_consultas_ajustado['Categoria'].str.contains('Odontológica')]
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,-
190,Vila Mariana,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,139951


Ainda, substituímos os `-` por 0 na coluna de quantidade de consultas.

In [14]:
hifen = df_consultas_ajustado['Qtd_Consultas'] == '-'
df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0
df_consultas_ajustado.loc[:, 'Qtd_Consultas'] = df_consultas_ajustado.loc[:, 'Qtd_Consultas'].astype(int)
df_consultas_ajustado

C:\Users\855395\AppData\Local\Temp\ipykernel_12928\3378969537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0


,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,139951


Por último, vamos corrigir os nomes das categorias.

In [15]:
df_consultas_ajustado['Categoria'].value_counts()

Categoria
Consulta Médica\nna Atenção Básica(2)                     64
Consulta Médica\nna Atenção Especializada                 64
Consulta Médica/Atendimento\nem Urgência/Emergência(3)    64
Name: count, dtype: int64

In [16]:
df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica na Atenção Básica' if 'Atenção Básica' in c else c)

df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica/Atendimento em Urgência/Emergência' if 'Urgência' in c else c)

df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951


## Leitos de internação e complementares

Os dados de leitos não estão disponíveis de maneira regionalizada no boletim "Saúde em dados", então terão de ser obtidos de outra fonte. Eles estão disponíveis na plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Entretanto, os links dos arquivos gerados nessa plataforma podem não ser permanentes, então utilizaremos uma versão já baixada do arquivo csv.

Obs: caso a necessidade fosse mais geral ou a atualização mais frequente, seria possível desenvolver um *scrapper* para a plataforma, mas estrapola o escopo deste trabalho.

In [17]:
data_path = path.join('dados', 'saude')
leitos_filename = 'A161721192_29_142_217.csv'
leitos_file_path = path.join(data_path, leitos_filename)

df_leito = pd.read_csv(leitos_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA,Total
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-,95.0
1,Butantã,78,131,66,57,70,8,30,440.0
2,Campo Limpo,-,-,-,-,-,88,7,95.0
3,Capela do Socorro,78,155,96,62,38,28,-,457.0
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4,387.0
5,Cidade Ademar,-,-,-,-,-,-,14,14.0
6,Cidade Tiradentes,49,54,45,38,27,10,1,224.0
7,Ermelino Matarazzo,85,58,44,40,29,-,-,256.0
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13,387.0
9,Guaianases,40,58,46,48,33,24,-,249.0


Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [18]:
df_leito = df_leito.iloc[:32]
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA,Total
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-,95.0
1,Butantã,78,131,66,57,70,8,30,440.0
2,Campo Limpo,-,-,-,-,-,88,7,95.0
3,Capela do Socorro,78,155,96,62,38,28,-,457.0
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4,387.0
5,Cidade Ademar,-,-,-,-,-,-,14,14.0
6,Cidade Tiradentes,49,54,45,38,27,10,1,224.0
7,Ermelino Matarazzo,85,58,44,40,29,-,-,256.0
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13,387.0
9,Guaianases,40,58,46,48,33,24,-,249.0


In [19]:
cols_total = df_leito.columns.str.lower().str.contains('total')
df_leito = df_leito.loc[:,~cols_total]
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-
1,Butantã,78,131,66,57,70,8,30
2,Campo Limpo,-,-,-,-,-,88,7
3,Capela do Socorro,78,155,96,62,38,28,-
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4
5,Cidade Ademar,-,-,-,-,-,-,14
6,Cidade Tiradentes,49,54,45,38,27,10,1
7,Ermelino Matarazzo,85,58,44,40,29,-,-
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13
9,Guaianases,40,58,46,48,33,24,-


Em seguida, "despivotamos" a tabela.

In [20]:
df_leito_ajustado = df_leito.melt(id_vars=['Pref Regional'],
                                    var_name='Categoria específica',
                                    value_name='Qtd leitos')
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,-
1,Butantã,1-Cirúrgico,78
2,Campo Limpo,1-Cirúrgico,-
3,Capela do Socorro,1-Cirúrgico,78
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86
...,...,...,...
219,Sapopemba,7-Hospital/DIA,6
220,Sé,7-Hospital/DIA,69
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6
222,Vila Mariana,7-Hospital/DIA,166


Por último, substituimos os `-` por 0.

In [21]:
hifen = df_leito_ajustado['Qtd leitos'].str.contains('-')
df_leito_ajustado.loc[hifen,'Qtd leitos'] = 0
df_leito_ajustado.loc[:,'Qtd leitos'] = df_leito_ajustado.loc[:,'Qtd leitos'].astype(int)
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0
1,Butantã,1-Cirúrgico,78
2,Campo Limpo,1-Cirúrgico,0
3,Capela do Socorro,1-Cirúrgico,78
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86
...,...,...,...
219,Sapopemba,7-Hospital/DIA,6
220,Sé,7-Hospital/DIA,69
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6
222,Vila Mariana,7-Hospital/DIA,166


Posteriormente, cada categoria específica dos leitos será categorizada em Leitos de internações, leitos complementares ou leitos de UTI, de acordo com as variáveis definidas.

## Mortalidade

Os dados sobre mortalidade também serão extraídos da plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Novamente, como os links dos arquivos gerados nessa plataforma podem não ser permanentes, utilizaremos uma versão já baixada do arquivo csv.

In [22]:
data_path = path.join('dados', 'saude')
mortalidade_filename = 'A180859192_29_141_183.csv'
mortalidade_file_path = path.join(data_path, mortalidade_filename)

df_mortalidade = pd.read_csv(mortalidade_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,CA colon,"CA reto, JRS, ânus",CA fígado,CA pâncreas,CA laringe,CA pulmão,CA mama,CA ovário,CA colo de utero,CA útero porção n/esp,CA próstata,CA encéfalo,Leucemias,Linfomas,Anemias,Diabetes mellitus,Desnutrição,Obesidade,Alcoolismo,Meningites,D. Alzheimer,Demência,D. reumática crônica coração,D. hipertensivas,Doenças isquêmicas coração,D. circulacao pulmonar,Miocardiopatias,Insuficiencia cardiaca,D. cerebrovasculares,Aneurisma e dissecção aorta,Pneumonias,"Bronquite, enfisema, asma",Outras doenças pulmão,Úlcera estômago e duodeno,Doença alcoólica do fígado,Fibrose e cirrose hepática (exceto alcóol.),Insuficiência renal,Infecção do trato urinário n/ espec,"Gravidez, parto e puerpério","Feto/RN afet fat matern e compl grav,parto",Hipóxia intra-uterina/asfixia nascer,D. membrana hialina,Outras afec. respiratórias RN,Infecções espec. periodo perinatal,Demais causas perinatais,Anom congenitas sist nervoso,Anom congenitas coracao e circ,Anom congenitas ap digestivo,Demais anomalias congênitas,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte,Total
0,Aricanduva/Formosa/Carrão,16.0,6.0,-,38.0,6.0,788.0,3,8,22.0,41.0,16.0,30.0,28.0,10.0,52.0,46.0,15,7.0,2,28,15.0,18,11,2,95.0,3,8.0,3,-,64.0,32.0,2,108.0,292.0,36.0,23.0,78.0,172.0,21.0,156.0,69.0,15.0,7.0,12.0,15.0,42.0,77.0,2,4,1,1,-,1,1,-,7.0,-,5,63.0,15.0,21.0,53.0,11.0,10.0,24.0,2,507.0,3266.0
1,Butantã,14.0,4.0,-,30.0,14.0,696.0,5,13,34.0,44.0,27.0,22.0,33.0,9.0,80.0,59.0,13,12.0,6,21,23.0,21,11,9,115.0,4,16.0,8,2,71.0,67.0,6,126.0,223.0,29.0,55.0,74.0,179.0,31.0,167.0,83.0,14.0,9.0,30.0,23.0,51.0,58.0,2,12,2,1,3,3,7,1,9.0,-,9,86.0,30.0,40.0,44.0,26.0,15.0,34.0,4,646.0,3615.0
2,Campo Limpo,13.0,10.0,-,43.0,20.0,1005.0,5,14,42.0,50.0,25.0,28.0,41.0,8.0,87.0,26.0,23,15.0,8,32,35.0,14,18,5,134.0,-,12.0,16,4,53.0,27.0,10,139.0,369.0,49.0,53.0,65.0,237.0,29.0,190.0,105.0,22.0,13.0,31.0,32.0,45.0,64.0,9,10,4,3,2,6,12,3,10.0,3,8,81.0,62.0,49.0,77.0,25.0,55.0,46.0,5,673.0,4409.0
3,Capela do Socorro,22.0,19.0,-,52.0,21.0,1070.0,9,11,44.0,62.0,33.0,20.0,43.0,18.0,73.0,57.0,14,19.0,4,40,20.0,20,12,5,145.0,6,12.0,7,1,45.0,24.0,4,147.0,448.0,50.0,40.0,82.0,298.0,31.0,197.0,86.0,16.0,14.0,43.0,36.0,62.0,63.0,8,17,4,5,9,15,7,3,10.0,2,13,109.0,51.0,61.0,71.0,17.0,46.0,31.0,4,793.0,4821.0
4,Casa Verde/Cachoeirinha,12.0,16.0,-,39.0,19.0,794.0,7,14,36.0,41.0,21.0,25.0,23.0,6.0,54.0,28.0,9,1.0,3,27,15.0,10,9,7,118.0,4,7.0,7,4,48.0,23.0,3,126.0,177.0,25.0,41.0,60.0,175.0,14.0,160.0,86.0,14.0,10.0,14.0,10.0,40.0,52.0,2,7,3,5,1,10,7,1,4.0,2,2,72.0,18.0,39.0,48.0,14.0,26.0,16.0,4,501.0,3216.0
5,Cidade Ademar,10.0,19.0,-,27.0,15.0,764.0,5,26,33.0,23.0,15.0,18.0,21.0,12.0,59.0,43.0,8,9.0,5,20,16.0,9,10,4,99.0,1,16.0,9,2,37.0,14.0,3,100.0,290.0,32.0,23.0,57.0,184.0,22.0,155.0,84.0,15.0,6.0,35.0,16.0,47.0,45.0,4,14,1,2,5,5,9,1,10.0,-,8,89.0,31.0,46.0,51.0,20.0,33.0,27.0,3,519.0,3341.0
6,Cidade Tiradentes,2.0,11.0,-,21.0,11.0,415.0,3,9,11.0,12.0,6.0,5.0,10.0,9.0,20.0,18.0,3,9.0,3,11,7.0,10,2,3,63.0,-,5.0,9,1,15.0,1.0,3,53.0,164.0,12.0,6.0,23.0,90.0,9.0,84.0,36.0,9.0,7.0,16.0,11.0,8.0,23.0,3,6,1,2,5,7,3,2,5.0,2,3,42.0,18.0,9.0,35.0,8.0,16.0,10.0,11,202.0,1649.0
7,Ermelino Matarazzo,11.0,8.0,-,19.0,10.0,488.0,2,8,22.0,28.0,9.0,7.0,24.0,6.0,29.0,26.0,4,4.0,3,23,5.0,6,7,3,48.0,1,8.0,4,2,28.0,11.0,2,65.0,186.0,30.0,14.0,49.0,114.0,11.0,95.0,47.0,10.0,9.0,14.0,12.0,27.0,41.0,4,4,1,-,-,2,2,-,4.0,1,5,56.0,27.0,18.0,29.0,10.0,13.0,12.0,7,334.0,2109.0
8,Freguesia/Brasilândia,3.0,10.0,-,41.0,13.0,985.0,9,16,27.0,34.0,16.0,24.0,20.0,10.0,60.0,47.0,9,8.0,4,28,17.0,15,7,5,153.0,7,12.0,10,2,48.0,30.0,4,149.0,235.0,45.0,53.0,89.0,230.0,24.0,190.0,83.0,15.0,14.0,28.0,20.0,50.0,67.0,5,6,2,1,6,8,13,1,9.0,-,7,134.0,32.0,43.0,67.0,19.0,33.0,27.0,4,599.0,3982.0


Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [23]:
df_mortalidade = df_mortalidade.iloc[:32]
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,CA colon,"CA reto, JRS, ânus",CA fígado,CA pâncreas,CA laringe,CA pulmão,CA mama,CA ovário,CA colo de utero,CA útero porção n/esp,CA próstata,CA encéfalo,Leucemias,Linfomas,Anemias,Diabetes mellitus,Desnutrição,Obesidade,Alcoolismo,Meningites,D. Alzheimer,Demência,D. reumática crônica coração,D. hipertensivas,Doenças isquêmicas coração,D. circulacao pulmonar,Miocardiopatias,Insuficiencia cardiaca,D. cerebrovasculares,Aneurisma e dissecção aorta,Pneumonias,"Bronquite, enfisema, asma",Outras doenças pulmão,Úlcera estômago e duodeno,Doença alcoólica do fígado,Fibrose e cirrose hepática (exceto alcóol.),Insuficiência renal,Infecção do trato urinário n/ espec,"Gravidez, parto e puerpério","Feto/RN afet fat matern e compl grav,parto",Hipóxia intra-uterina/asfixia nascer,D. membrana hialina,Outras afec. respiratórias RN,Infecções espec. periodo perinatal,Demais causas perinatais,Anom congenitas sist nervoso,Anom congenitas coracao e circ,Anom congenitas ap digestivo,Demais anomalias congênitas,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte,Total
0,Aricanduva/Formosa/Carrão,16.0,6.0,-,38.0,6.0,788.0,3,8,22.0,41.0,16.0,30.0,28.0,10.0,52.0,46.0,15,7.0,2,28,15.0,18,11,2,95.0,3,8.0,3,-,64.0,32.0,2,108.0,292.0,36.0,23.0,78.0,172.0,21.0,156.0,69.0,15.0,7.0,12.0,15.0,42.0,77.0,2,4,1,1,-,1,1,-,7.0,-,5,63.0,15.0,21.0,53.0,11.0,10.0,24.0,2,507.0,3266.0
1,Butantã,14.0,4.0,-,30.0,14.0,696.0,5,13,34.0,44.0,27.0,22.0,33.0,9.0,80.0,59.0,13,12.0,6,21,23.0,21,11,9,115.0,4,16.0,8,2,71.0,67.0,6,126.0,223.0,29.0,55.0,74.0,179.0,31.0,167.0,83.0,14.0,9.0,30.0,23.0,51.0,58.0,2,12,2,1,3,3,7,1,9.0,-,9,86.0,30.0,40.0,44.0,26.0,15.0,34.0,4,646.0,3615.0
2,Campo Limpo,13.0,10.0,-,43.0,20.0,1005.0,5,14,42.0,50.0,25.0,28.0,41.0,8.0,87.0,26.0,23,15.0,8,32,35.0,14,18,5,134.0,-,12.0,16,4,53.0,27.0,10,139.0,369.0,49.0,53.0,65.0,237.0,29.0,190.0,105.0,22.0,13.0,31.0,32.0,45.0,64.0,9,10,4,3,2,6,12,3,10.0,3,8,81.0,62.0,49.0,77.0,25.0,55.0,46.0,5,673.0,4409.0
3,Capela do Socorro,22.0,19.0,-,52.0,21.0,1070.0,9,11,44.0,62.0,33.0,20.0,43.0,18.0,73.0,57.0,14,19.0,4,40,20.0,20,12,5,145.0,6,12.0,7,1,45.0,24.0,4,147.0,448.0,50.0,40.0,82.0,298.0,31.0,197.0,86.0,16.0,14.0,43.0,36.0,62.0,63.0,8,17,4,5,9,15,7,3,10.0,2,13,109.0,51.0,61.0,71.0,17.0,46.0,31.0,4,793.0,4821.0
4,Casa Verde/Cachoeirinha,12.0,16.0,-,39.0,19.0,794.0,7,14,36.0,41.0,21.0,25.0,23.0,6.0,54.0,28.0,9,1.0,3,27,15.0,10,9,7,118.0,4,7.0,7,4,48.0,23.0,3,126.0,177.0,25.0,41.0,60.0,175.0,14.0,160.0,86.0,14.0,10.0,14.0,10.0,40.0,52.0,2,7,3,5,1,10,7,1,4.0,2,2,72.0,18.0,39.0,48.0,14.0,26.0,16.0,4,501.0,3216.0
5,Cidade Ademar,10.0,19.0,-,27.0,15.0,764.0,5,26,33.0,23.0,15.0,18.0,21.0,12.0,59.0,43.0,8,9.0,5,20,16.0,9,10,4,99.0,1,16.0,9,2,37.0,14.0,3,100.0,290.0,32.0,23.0,57.0,184.0,22.0,155.0,84.0,15.0,6.0,35.0,16.0,47.0,45.0,4,14,1,2,5,5,9,1,10.0,-,8,89.0,31.0,46.0,51.0,20.0,33.0,27.0,3,519.0,3341.0
6,Cidade Tiradentes,2.0,11.0,-,21.0,11.0,415.0,3,9,11.0,12.0,6.0,5.0,10.0,9.0,20.0,18.0,3,9.0,3,11,7.0,10,2,3,63.0,-,5.0,9,1,15.0,1.0,3,53.0,164.0,12.0,6.0,23.0,90.0,9.0,84.0,36.0,9.0,7.0,16.0,11.0,8.0,23.0,3,6,1,2,5,7,3,2,5.0,2,3,42.0,18.0,9.0,35.0,8.0,16.0,10.0,11,202.0,1649.0
7,Ermelino Matarazzo,11.0,8.0,-,19.0,10.0,488.0,2,8,22.0,28.0,9.0,7.0,24.0,6.0,29.0,26.0,4,4.0,3,23,5.0,6,7,3,48.0,1,8.0,4,2,28.0,11.0,2,65.0,186.0,30.0,14.0,49.0,114.0,11.0,95.0,47.0,10.0,9.0,14.0,12.0,27.0,41.0,4,4,1,-,-,2,2,-,4.0,1,5,56.0,27.0,18.0,29.0,10.0,13.0,12.0,7,334.0,2109.0
8,Freguesia/Brasilândia,3.0,10.0,-,41.0,13.0,985.0,9,16,27.0,34.0,16.0,24.0,20.0,10.0,60.0,47.0,9,8.0,4,28,17.0,15,7,5,153.0,7,12.0,10,2,48.0,30.0,4,149.0,235.0,45.0,53.0,89.0,230.0,24.0,190.0,83.0,15.0,14.0,28.0,20.0,50.0,67.0,5,6,2,1,6,8,13,1,9.0,-,7,134.0,32.0,43.0,67.0,19.0,33.0,27.0,4,599.0,3982.0


In [24]:
cols_total = df_mortalidade.columns.str.lower().str.contains('total')
df_mortalidade = df_mortalidade.loc[:,~cols_total]
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,CA colon,"CA reto, JRS, ânus",CA fígado,CA pâncreas,CA laringe,CA pulmão,CA mama,CA ovário,CA colo de utero,CA útero porção n/esp,CA próstata,CA encéfalo,Leucemias,Linfomas,Anemias,Diabetes mellitus,Desnutrição,Obesidade,Alcoolismo,Meningites,D. Alzheimer,Demência,D. reumática crônica coração,D. hipertensivas,Doenças isquêmicas coração,D. circulacao pulmonar,Miocardiopatias,Insuficiencia cardiaca,D. cerebrovasculares,Aneurisma e dissecção aorta,Pneumonias,"Bronquite, enfisema, asma",Outras doenças pulmão,Úlcera estômago e duodeno,Doença alcoólica do fígado,Fibrose e cirrose hepática (exceto alcóol.),Insuficiência renal,Infecção do trato urinário n/ espec,"Gravidez, parto e puerpério","Feto/RN afet fat matern e compl grav,parto",Hipóxia intra-uterina/asfixia nascer,D. membrana hialina,Outras afec. respiratórias RN,Infecções espec. periodo perinatal,Demais causas perinatais,Anom congenitas sist nervoso,Anom congenitas coracao e circ,Anom congenitas ap digestivo,Demais anomalias congênitas,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte
0,Aricanduva/Formosa/Carrão,16.0,6.0,-,38.0,6.0,788.0,3,8,22.0,41.0,16.0,30.0,28.0,10.0,52.0,46.0,15,7.0,2,28,15.0,18,11,2,95.0,3,8.0,3,-,64.0,32.0,2,108.0,292.0,36.0,23.0,78.0,172.0,21.0,156.0,69.0,15.0,7.0,12.0,15.0,42.0,77.0,2,4,1,1,-,1,1,-,7.0,-,5,63.0,15.0,21.0,53.0,11.0,10.0,24.0,2,507.0
1,Butantã,14.0,4.0,-,30.0,14.0,696.0,5,13,34.0,44.0,27.0,22.0,33.0,9.0,80.0,59.0,13,12.0,6,21,23.0,21,11,9,115.0,4,16.0,8,2,71.0,67.0,6,126.0,223.0,29.0,55.0,74.0,179.0,31.0,167.0,83.0,14.0,9.0,30.0,23.0,51.0,58.0,2,12,2,1,3,3,7,1,9.0,-,9,86.0,30.0,40.0,44.0,26.0,15.0,34.0,4,646.0
2,Campo Limpo,13.0,10.0,-,43.0,20.0,1005.0,5,14,42.0,50.0,25.0,28.0,41.0,8.0,87.0,26.0,23,15.0,8,32,35.0,14,18,5,134.0,-,12.0,16,4,53.0,27.0,10,139.0,369.0,49.0,53.0,65.0,237.0,29.0,190.0,105.0,22.0,13.0,31.0,32.0,45.0,64.0,9,10,4,3,2,6,12,3,10.0,3,8,81.0,62.0,49.0,77.0,25.0,55.0,46.0,5,673.0
3,Capela do Socorro,22.0,19.0,-,52.0,21.0,1070.0,9,11,44.0,62.0,33.0,20.0,43.0,18.0,73.0,57.0,14,19.0,4,40,20.0,20,12,5,145.0,6,12.0,7,1,45.0,24.0,4,147.0,448.0,50.0,40.0,82.0,298.0,31.0,197.0,86.0,16.0,14.0,43.0,36.0,62.0,63.0,8,17,4,5,9,15,7,3,10.0,2,13,109.0,51.0,61.0,71.0,17.0,46.0,31.0,4,793.0
4,Casa Verde/Cachoeirinha,12.0,16.0,-,39.0,19.0,794.0,7,14,36.0,41.0,21.0,25.0,23.0,6.0,54.0,28.0,9,1.0,3,27,15.0,10,9,7,118.0,4,7.0,7,4,48.0,23.0,3,126.0,177.0,25.0,41.0,60.0,175.0,14.0,160.0,86.0,14.0,10.0,14.0,10.0,40.0,52.0,2,7,3,5,1,10,7,1,4.0,2,2,72.0,18.0,39.0,48.0,14.0,26.0,16.0,4,501.0
5,Cidade Ademar,10.0,19.0,-,27.0,15.0,764.0,5,26,33.0,23.0,15.0,18.0,21.0,12.0,59.0,43.0,8,9.0,5,20,16.0,9,10,4,99.0,1,16.0,9,2,37.0,14.0,3,100.0,290.0,32.0,23.0,57.0,184.0,22.0,155.0,84.0,15.0,6.0,35.0,16.0,47.0,45.0,4,14,1,2,5,5,9,1,10.0,-,8,89.0,31.0,46.0,51.0,20.0,33.0,27.0,3,519.0
6,Cidade Tiradentes,2.0,11.0,-,21.0,11.0,415.0,3,9,11.0,12.0,6.0,5.0,10.0,9.0,20.0,18.0,3,9.0,3,11,7.0,10,2,3,63.0,-,5.0,9,1,15.0,1.0,3,53.0,164.0,12.0,6.0,23.0,90.0,9.0,84.0,36.0,9.0,7.0,16.0,11.0,8.0,23.0,3,6,1,2,5,7,3,2,5.0,2,3,42.0,18.0,9.0,35.0,8.0,16.0,10.0,11,202.0
7,Ermelino Matarazzo,11.0,8.0,-,19.0,10.0,488.0,2,8,22.0,28.0,9.0,7.0,24.0,6.0,29.0,26.0,4,4.0,3,23,5.0,6,7,3,48.0,1,8.0,4,2,28.0,11.0,2,65.0,186.0,30.0,14.0,49.0,114.0,11.0,95.0,47.0,10.0,9.0,14.0,12.0,27.0,41.0,4,4,1,-,-,2,2,-,4.0,1,5,56.0,27.0,18.0,29.0,10.0,13.0,12.0,7,334.0
8,Freguesia/Brasilândia,3.0,10.0,-,41.0,13.0,985.0,9,16,27.0,34.0,16.0,24.0,20.0,10.0,60.0,47.0,9,8.0,4,28,17.0,15,7,5,153.0,7,12.0,10,2,48.0,30.0,4,149.0,235.0,45.0,53.0,89.0,230.0,24.0,190.0,83.0,15.0,14.0,28.0,20.0,50.0,67.0,5,6,2,1,6,8,13,1,9.0,-,7,134.0,32.0,43.0,67.0,19.0,33.0,27.0,4,599.0
9,Guaianases,10.0,14.0,1,38.0,16.0,510.0,3,11,13.0,21.0,6.0,16.0,9.0,

Em seguida, "despivotamos" a tabela.

In [25]:
df_mortalidade_ajustado = df_mortalidade.melt(id_vars=['Subprefeitura residência'],
                                    var_name='Causa específica',
                                    value_name='Qtd óbitos')
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16.0
1,Butantã,Doencas infec intest,14.0
2,Campo Limpo,Doencas infec intest,13.0
3,Capela do Socorro,Doencas infec intest,22.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,12.0
...,...,...,...
2139,Sapopemba,Demais causas de morte,429.0
2140,Sé,Demais causas de morte,733.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523.0
2142,Vila Mariana,Demais causas de morte,682.0


Por último, substituimos os `-` por 0.

In [26]:
hifen = df_mortalidade_ajustado['Qtd óbitos'].astype(str).str.contains('-')
df_mortalidade_ajustado.loc[hifen,'Qtd óbitos'] = 0
df_mortalidade_ajustado.loc[:,'Qtd óbitos'] = df_mortalidade_ajustado.loc[:,'Qtd óbitos'].astype(int)
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16
1,Butantã,Doencas infec intest,14
2,Campo Limpo,Doencas infec intest,13
3,Capela do Socorro,Doencas infec intest,22
4,Casa Verde/Cachoeirinha,Doencas infec intest,12
...,...,...,...
2139,Sapopemba,Demais causas de morte,429
2140,Sé,Demais causas de morte,733
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523
2142,Vila Mariana,Demais causas de morte,682


Posteriormente, cada causa específica dos óbitos será categorizado em Evitável, Tratável ou Evitável/Tratável, de acordo com uma tabela auxiliar definida pelo GT.

## Despesas

Para que seja possível relacionar as despesas com os outros indicadores, podemos usar os dados de regionalização do orçamento, disponíveis de acordo com o valor de detalhamento da ação.

Esses dados ficam disponíveis na [página de prestação de contas públicas da Secretaria Municipal da Fazenda](https://orcamento.sf.prefeitura.sp.gov.br/orcamento/execucao.php), na Base de Dados da Regionalização da Execução Orçamentária.

In [27]:
despesas_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2023/basedadosDA_1223.csv'
df_despesa = pd.read_csv(despesas_url,
                         encoding='latin1',
                         sep=';',
                         decimal=',')

df_despesa

C:\Users\855395\AppData\Local\Temp\ipykernel_12928\4118471245.py:2: DtypeWarning: Columns (27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_despesa = pd.read_csv(despesas_url,


,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,DESCRIÇÃO_UNIDADE,CÓDIGO_FUNÇÃO,DESCRIÇÃO_FUNÇÃO,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,CÓDIGO_PROJ_ATIV,DESCRIÇÃO_PROJ_ATIV,CÓDIGO_CONTA_DESPESA,DESCRIÇÃO_CONTA_DESPESA,CÓDIGO_FONTE,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,2305,2023,6655,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",33903000,Material de Consumo,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,4850.00,25/06/2025
1,4,2389,2023,39,2024,2024-01-18 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,126348.08,25/06/2025
2,4,2389,2023,6385,2023,2023-11-10 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,150000.00,25/06/2025
3,4,2389,2023,6654,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,223651.92,25/06/2025
4,4,3280,2023,120,2024,2024-01-19 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,19139.06,25/06/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476485,1,110546,2023,30319,2024,2024-02-06 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,15,Urbanismo,452,Serviços Urbanos,3022,Requalificação e Promoção da Ocupação dos Espa...,6027,Contraprestação de Parceria Público-Privada (P...,33678300,Despesas Decorrentes de Contrato de Parceria P...,8,PMSP-SIURB/Fundo Municipal de Iluminação Públi...,1,751,253,0,NaN,NaN,NaN,NaN,NaN,25/06/2025
476486,1,110546,2023,30335,2024,2024-02-06 00:00:00.0000000,99,FUNDIP,Fundo Municipal de Iluminação Pública,10,Fundo Municipal de Iluminação Pública,15,Urbanismo,452,Serviços Urbanos,3022,Requalificação e Promoção da Ocupação dos Espa...,6027,Contraprestação de Parceria Público-Privada (P...,33678300,Despesas Decorrentes de Contrato de Parceria P...,8,PMSP-SIURB/Fundo Municipal de Iluminação Públi...,1,751,253,0,NaN,NaN,NaN,NaN,NaN,25/06/2025
476487,1,110547,2023,30339,2024,2024-02

Primeiro, vamos filtrar os dados para manter apenas a função Saúde do orçamento.

In [28]:
df_despesa = df_despesa[df_despesa['DESCRIÇÃO_FUNÇÃO']=='Saúde']
df_despesa

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,DESCRIÇÃO_UNIDADE,CÓDIGO_FUNÇÃO,DESCRIÇÃO_FUNÇÃO,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,CÓDIGO_PROJ_ATIV,DESCRIÇÃO_PROJ_ATIV,CÓDIGO_CONTA_DESPESA,DESCRIÇÃO_CONTA_DESPESA,CÓDIGO_FONTE,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,2305,2023,6655,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",33903000,Material de Consumo,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,4850.00,25/06/2025
1,4,2389,2023,39,2024,2024-01-18 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,126348.08,25/06/2025
2,4,2389,2023,6385,2023,2023-11-10 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,150000.00,25/06/2025
3,4,2389,2023,6654,2023,2023-11-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,223651.92,25/06/2025
4,4,3280,2023,120,2024,2024-01-19 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",2818,"Aquisição de Materiais, Equipamentos e Serviço...",44904000,Serviços de Tecnologia da Informação e Comunic...,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,19139.06,25/06/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434586,1,105049,2023,18117,2024,2024-01-24 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,28,Coordenadoria Regional de Saúde Centro,10,Saúde,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2519,"Manutenção e Operação em Atenção Básica, Espec...",33903000,Material de Consumo,0,Recursos não vinculados de Impostos,1,500,9001,1,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,2246.25,25/06/2025
434587,1,107178,2023,18126,2024,2024-01-24 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,28,Coordenadoria Regional de Saúde Centro,10,Saúde,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2519,"Manutenção e Operação em Atenção Básica, Espec...",33903000,Material de Consumo,0,Recursos não vinculados de Impostos,1,500,9001,1,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,78.78,25/06/2025
434588,1,1071

In [29]:
group_cols = ['CÓDIGO_SUBFUNÇÃO',
        'DESCRIÇÃO_SUBFUNÇÃO',
        'CÓDIGO_PROGRAMA',
        'DESCRIÇÃO_PROGRAMA',
        'REGIÃO',
        'SUBPREFEITURA',
        'DISTRITO',
        'TIPO_REGIONALIZAÇÃO']

metric_cols = ['VALOR_DETALHAMENTO_AÇÃO']

df_despesa_agrupado = df_despesa.groupby(group_cols).sum().reset_index().loc[:, group_cols + metric_cols]
df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09
...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05


Vamos ver quais valores aparecem como subfunção.

In [30]:
df_despesa_agrupado['DESCRIÇÃO_PROGRAMA'].value_counts()

DESCRIÇÃO_PROGRAMA
Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância      88
Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência    38
Ações e Serviços da Saúde Animal                                              10
Suporte Administrativo                                                         4
Modernização Tecnológica, Desburocratização e Inovação do Serviço Público      3
Benefícios e Previdência de Funcionários                                       1
Participação, Transparência e Controle Social da Administração Pública         1
Name: count, dtype: int64

Para os indicadores de "Despesas liquidadas em atenção básica e vigilância" e "Despesas liquidadas em atenção especializada e Suporte profilático terapêutico", parece que os programas serão o suficiente, com alguns não se encaixando em nenhuma das categorias.

Baixados todos os dados, vamos começar as transformações para adequar as categorias dos dados às necessidades dos indicadores.

# Transformação e padronização

Primeiro, vamos criar as variáveis que representarão os níveis de atenção padronizados.

In [31]:
NIVEL_COL_NAME = 'Nível de atenção padronizado'
NIVEL_AB_VIGILANCIA = 'Atenção Básica e Vigilância'
NIVEL_ESP = 'Atenção Especializada e Suporte profilático terapêutico'
NIVEL_OUTROS = 'Outros níveis de atenção'

Nas próximas seções, vamos também padronizar os nomes das Subprefeituras, com uma função já disponível no módulo `core.utils.cleaning`.

## Consultas médicas

In [32]:
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951


In [33]:
subpref_map = {
    "Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
    "Butantã": "BUTANTA",
    "Campo Limpo": "CAMPO LIMPO",
    "Capela do Socorro": "CAPELA DO SOCORRO",
    "Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
    "Cidade Ademar": "CIDADE ADEMAR",
    "Cidade Tiradentes": "CIDADE TIRADENTES",
    "Ermelino Matarazzo": "ERMELINO MATARAZZO",
    "Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
    "Guaianases": "GUAIANASES",
    "Ipiranga": "IPIRANGA",
    "Itaim Paulista": "ITAIM PAULISTA",
    "Itaquera": "ITAQUERA",
    "Jabaquara": "JABAQUARA",
    "Jaçanã/Tremembé": "JACANA-TREMEMBE",
    "Lapa": "LAPA",
    "MBoi Mirim": "M BOI MIRIM",
    "Mooca": "MOOCA",
    "Parelheiros": "PARELEIROS",
    "Penha": "PENHA",
    "Perus": "PERUS",
    "Pinheiros": "PINHEIROS",
    "Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
    "Santana/Tucuruvi": "SANTANA-TUCURUVI",
    "Santo Amaro": "SANTO AMARO",
    "São Mateus": "SAO MATEUS",
    "São Miguel Paulista": "SAO MIGUEL",
    "Sapopemba": "SAPOPEMBA",
    "Sé": "SE",
    "Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
    "Vila Mariana": "VILA MARIANA",
    "Vila Prudente": "VILA PRUDENTE"
}

In [34]:
df_consultas_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (
    df_consultas_ajustado.loc[:, 'Subprefeitura'].map(subpref_map)
)

df_consultas_ajustado

C:\Users\855395\AppData\Local\Temp\ipykernel_12928\2884928701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consultas_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (


,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas,SUBPREFEITURA_QLIK
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894,ARICANDUVA-FORMOSA-CARRAO
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917,BUTANTA
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251,CAMPO LIMPO
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063,CAPELA DO SOCORRO
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363,CASA VERDE-CACHOEIRINHA
...,...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822,SAPOPEMBA
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857,SE
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0,VILA MARIA-VILA GUILHERME
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951,VILA MARIANA


In [35]:
df_consultas_ajustado['Categoria'].unique()

array(['Consulta Médica na Atenção Básica',
       'Consulta Médica\nna Atenção Especializada',
       'Consulta Médica/Atendimento em Urgência/Emergência'], dtype=object)

In [36]:
df_consultas_final = df_consultas_ajustado.copy()

filtro_ab = df_consultas_final['Categoria'].str.contains('Atenção Básica')

df_consultas_final.loc[filtro_ab, NIVEL_COL_NAME] = NIVEL_AB_VIGILANCIA
df_consultas_final.loc[~filtro_ab, NIVEL_COL_NAME] = NIVEL_ESP

df_consultas_final

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas,SUBPREFEITURA_QLIK,Nível de atenção padronizado
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894,ARICANDUVA-FORMOSA-CARRAO,Atenção Básica e Vigilância
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917,BUTANTA,Atenção Básica e Vigilância
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251,CAMPO LIMPO,Atenção Básica e Vigilância
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063,CAPELA DO SOCORRO,Atenção Básica e Vigilância
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363,CASA VERDE-CACHOEIRINHA,Atenção Básica e Vigilância
...,...,...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822,SAPOPEMBA,Atenção Especializada e Suporte profilático te...
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857,SE,Atenção Especializada e Suporte profilático te...
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0,VILA MARIA-VILA GUILHERME,Atenção Especializada e Suporte profilático te...
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951,VILA MARIANA,Atenção Especializada e Suporte profilático te...


## Leitos de internação e complementares

Primeiro, vamos organizar os leitos nas categorias: Leitos de internações; Leitos complementares.

A regra específicada pelo GT foi a seguinte:

 - Leitos de internações são todas as categorias de leito, exceto as categorias `3-Complementar` e `7-Hospital/DIA`;

 - Leitos complementares são os da categoria `3-Complementar`;

In [37]:
df_leito_ajustado['Categoria específica'].value_counts()

Categoria específica
1-Cirúrgico                32
2-Clínico                  32
3-Complementar             32
4-Obstétrico               32
5-Pediátrico               32
6-Outras Especialidades    32
7-Hospital/DIA             32
Name: count, dtype: int64

In [38]:
leito_map = {
    '1-Cirúrgico': 'Leitos de internação',
    '2-Clínico': 'Leitos de internação',
    '3-Complementar': 'Leitos complementares',
    '4-Obstétrico': 'Leitos de internação',
    '5-Pediátrico': 'Leitos de internação',
    '6-Outras Especialidades': 'Leitos de internação',
    '7-Hospital/DIA': None,
}

df_leito_ajustado.loc[:, 'Categoria geral'] = df_leito_ajustado['Categoria específica'].map(leito_map)
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos,Categoria geral
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0,Leitos de internação
1,Butantã,1-Cirúrgico,78,Leitos de internação
2,Campo Limpo,1-Cirúrgico,0,Leitos de internação
3,Capela do Socorro,1-Cirúrgico,78,Leitos de internação
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86,Leitos de internação
...,...,...,...,...
219,Sapopemba,7-Hospital/DIA,6,None
220,Sé,7-Hospital/DIA,69,None
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6,None
222,Vila Mariana,7-Hospital/DIA,166,None


Agora, adicionamos também a classificação geral por nível de atenção (básica ou especializada). Para leitos, todos são utilizados na Atenção especializada, então é só adicionarmos uma coluna com o valor como padrão.

In [39]:
df_leito_ajustado.loc[:, NIVEL_COL_NAME] = NIVEL_ESP
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos,Categoria geral,Nível de atenção padronizado
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0,Leitos de internação,Atenção Especializada e Suporte profilático te...
1,Butantã,1-Cirúrgico,78,Leitos de internação,Atenção Especializada e Suporte profilático te...
2,Campo Limpo,1-Cirúrgico,0,Leitos de internação,Atenção Especializada e Suporte profilático te...
3,Capela do Socorro,1-Cirúrgico,78,Leitos de internação,Atenção Especializada e Suporte profilático te...
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86,Leitos de internação,Atenção Especializada e Suporte profilático te...
...,...,...,...,...,...
219,Sapopemba,7-Hospital/DIA,6,None,Atenção Especializada e Suporte profilático te...
220,Sé,7-Hospital/DIA,69,None,Atenção Especializada e Suporte profilático te...
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6,None,Atenção Especializada e Suporte profilático te...
222,Vila Mariana,7-Hospital/DIA,166,None,Atenção Especializada e Suporte profilático te...


In [40]:
group_cols = ['Pref Regional',
              'Categoria geral',
              NIVEL_COL_NAME]

df_leitos_final = (
    df_leito_ajustado[group_cols + ['Qtd leitos']]
    .groupby(group_cols)
    .sum()
    .reset_index()
)

df_leitos_final

,Pref Regional,Categoria geral,Nível de atenção padronizado,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55


In [41]:
df_leitos_final = df_leitos_final.rename(columns={'Pref Regional': 'Subprefeitura'})
df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55


In [42]:
subpref_map = {
    "Aricanduva/ Formosa/ Carrão": "ARICANDUVA-FORMOSA-CARRAO",
    "Butantã": "BUTANTA",
    "Campo Limpo": "CAMPO LIMPO",
    "Capela do Socorro": "CAPELA DO SOCORRO",
    "Casa Verde/ Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
    "Cidade Ademar": "CIDADE ADEMAR",
    "Cidade Tiradentes": "CIDADE TIRADENTES",
    "Ermelino Matarazzo": "ERMELINO MATARAZZO",
    "Freguesia/ Brasilândia": "FREGUESIA-BRASILANDIA",
    "Guaianases": "GUAIANASES",
    "Ipiranga": "IPIRANGA",
    "Itaim Paulista": "ITAIM PAULISTA",
    "Itaquera": "ITAQUERA",
    "Jabaquara": "JABAQUARA",
    "Jaçanã/ Tremembé": "JACANA-TREMEMBE",
    "Lapa": "LAPA",
    "M Boi Mirim": "M BOI MIRIM",
    "Mooca": "MOOCA",
    "Parelheiros": "PARELEIROS",
    "Penha": "PENHA",
    "Perus": "PERUS",
    "Pinheiros": "PINHEIROS",
    "Pirituba/ Jaraguá": "PIRITUBA-JARAGUA",
    "Santana/ Tucuruvi": "SANTANA-TUCURUVI",
    "Santo Amaro": "SANTO AMARO",
    "Sapopemba": "SAPOPEMBA",
    "São Mateus": "SAO MATEUS",
    "São Miguel": "SAO MIGUEL",
    "Sé": "SE",
    "Vila Maria/ Vila Guilherme": "VILA MARIA-VILA GUILHERME",
    "Vila Mariana": "VILA MARIANA",
    "Vila Prudente": "VILA PRUDENTE"
}


In [43]:
df_leitos_final.loc[:, 'SUBPREFEITURA_QLIK'] = (
    df_leitos_final.loc[:, 'Subprefeitura'].map(subpref_map)
)

df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos,SUBPREFEITURA_QLIK
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,ARICANDUVA-FORMOSA-CARRAO
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95,ARICANDUVA-FORMOSA-CARRAO
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66,BUTANTA
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344,BUTANTA
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,CAMPO LIMPO
...,...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144,VILA MARIA-VILA GUILHERME
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324,VILA MARIANA
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491,VILA MARIANA
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55,VILA PRUDENTE


## Despesas

Para manter mais flexibilidade, vamos manter todas as colunas presentes na tabela de despesas, mesmo que usemos apenas a coluna de programa. Então, o que vamos fazer aqui é criar a nova coluna de nível de atenção padronizado com base nos programas.

In [44]:
df_despesa_agrupado['DESCRIÇÃO_PROGRAMA'].value_counts()

DESCRIÇÃO_PROGRAMA
Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância      88
Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência    38
Ações e Serviços da Saúde Animal                                              10
Suporte Administrativo                                                         4
Modernização Tecnológica, Desburocratização e Inovação do Serviço Público      3
Benefícios e Previdência de Funcionários                                       1
Participação, Transparência e Controle Social da Administração Pública         1
Name: count, dtype: int64

In [45]:
map_despesa = {
    'Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância': NIVEL_AB_VIGILANCIA,
    'Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência': NIVEL_ESP
}

df_despesa_agrupado.loc[:, NIVEL_COL_NAME] = df_despesa_agrupado.loc[:, 'DESCRIÇÃO_PROGRAMA'].map(map_despesa)
df_despesa_agrupado.loc[:, NIVEL_COL_NAME] = df_despesa_agrupado.loc[:, NIVEL_COL_NAME].fillna(NIVEL_OUTROS)
df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,Nível de atenção padronizado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07,Atenção Básica e Vigilância
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06,Outros níveis de atenção
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06,Outros níveis de atenção
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08,Outros níveis de atenção
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09,Outros níveis de atenção
...,...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05,Outros níveis de atenção
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06,Outros níveis de atenção
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06,Outros níveis de atenção
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05,Outros níveis de atenção


Vamos também criar uma nova coluna com o nome da subprefeitura, de acordo com o padrão existente no arquivo com os polígonos do Qlik.

In [46]:
subpref_filter = (
    df_despesa_agrupado.loc[:, 'SUBPREFEITURA']
    .str.startswith('Subprefeitura ')
)

df_despesa_agrupado.loc[subpref_filter, 'SUBPREFEITURA_QLIK'] = (
    df_despesa_agrupado
    .loc[subpref_filter, 'SUBPREFEITURA']
    .str.removeprefix('Subprefeitura ')
)

df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,Nível de atenção padronizado,SUBPREFEITURA_QLIK
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07,Atenção Básica e Vigilância,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06,Outros níveis de atenção,Sé
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06,Outros níveis de atenção,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08,Outros níveis de atenção,Sé
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09,Outros níveis de atenção,NaN
...,...,...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05,Outros níveis de atenção,Santana/Tucuruvi
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06,Outros níveis de atenção,Butantã
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06,Outros níveis de atenção,Pinheiros
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05,Outros níveis de atenção,Santo Amaro


In [47]:
subpref_map = {
    'Aricanduva/Formosa/Carrão': 'ARICANDUVA-FORMOSA-CARRAO',
    'Butantã': 'BUTANTA',
    'Campo Limpo': 'CAMPO LIMPO',
    'Capela do Socorro': 'CAPELA DO SOCORRO',
    'Casa Verde/Cachoeirinha': 'CASA VERDE-CACHOEIRINHA',
    'Cidade Ademar': 'CIDADE ADEMAR',
    'Cidade Tiradentes': 'CIDADE TIRADENTES',
    'Ermelino Matarazzo': 'ERMELINO MATARAZZO',
    'Freguesia/Brasilândia': 'FREGUESIA-BRASILANDIA',
    'Ipiranga': 'IPIRANGA',
    'Itaim Paulista': 'ITAIM PAULISTA',
    'Itaquera': 'ITAQUERA',
    'Jabaquara': 'JABAQUARA',
    'Jaçanã/Tremembé': 'JACANA-TREMEMBE',
    'Lapa': 'LAPA',
    "M'Boi Mirim": 'M BOI MIRIM',
    'Mooca': 'MOOCA',
    'Parelheiros': 'PARELEIROS',
    'Penha': 'PENHA',
    'Perus/Anhanguera': 'PERUS-ANHANGUERA',
    'Pinheiros': 'PINHEIROS',
    'Pirituba/Jaraguá': 'PIRITUBA-JARAGUA',
    'Santana/Tucuruvi': 'SANTANA-TUCURUVI',
    'Santo Amaro': 'SANTO AMARO',
    'Sapopemba': 'SAPOPEMBA',
    'São Mateus': 'SAO MATEUS',
    'São Miguel Paulista': 'SAO MIGUEL',
    'Sé': 'SE',
    'Vila Maria/Vila Guilherme': 'VILA MARIA-VILA GUILHERME',
    'Vila Mariana': 'VILA MARIANA',
    'de Guaianases': 'GUAIANASES',
    'de Vila Prudente': 'VILA PRUDENTE'
}

In [48]:
df_despesa_agrupado.loc[subpref_filter, 'SUBPREFEITURA_QLIK'] = (
    df_despesa_agrupado.loc[subpref_filter, 'SUBPREFEITURA_QLIK'].map(subpref_map)
)

df_despesa_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,Nível de atenção padronizado,SUBPREFEITURA_QLIK
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.736621e+07,Atenção Básica e Vigilância,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.371420e+06,Outros níveis de atenção,SE
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.907285e+06,Outros níveis de atenção,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228320e+08,Outros níveis de atenção,SE
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.826543e+09,Outros níveis de atenção,NaN
...,...,...,...,...,...,...,...,...,...,...,...
140,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,8.224286e+05,Outros níveis de atenção,SANTANA-TUCURUVI
141,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06,Outros níveis de atenção,BUTANTA
142,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06,Outros níveis de atenção,PINHEIROS
143,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05,Outros níveis de atenção,SANTO AMARO


## Mortalidade

In [49]:
subpref_map = {
    'Aricanduva/Formosa/Carrão': 'ARICANDUVA-FORMOSA-CARRAO',
    'Butantã': 'BUTANTA',
    'Campo Limpo': 'CAMPO LIMPO',
    'Capela do Socorro': 'CAPELA DO SOCORRO',
    'Casa Verde/Cachoeirinha': 'CASA VERDE-CACHOEIRINHA',
    'Cidade Ademar': 'CIDADE ADEMAR',
    'Cidade Tiradentes': 'CIDADE TIRADENTES',
    'Ermelino Matarazzo': 'ERMELINO MATARAZZO',
    'Freguesia/Brasilândia': 'FREGUESIA-BRASILANDIA',
    'Guaianases': 'GUAIANASES',
    'Ipiranga': 'IPIRANGA',
    'Itaim Paulista': 'ITAIM PAULISTA',
    'Itaquera': 'ITAQUERA',
    'Jabaquara': 'JABAQUARA',
    'Jaçanã/Tremembé': 'JACANA-TREMEMBE',
    'Lapa': 'LAPA',
    "M'Boi Mirim": 'M BOI MIRIM',
    'Moóca': 'MOOCA',
    'Parelheiros': 'PARELEIROS',
    'Penha': 'PENHA',
    'Perus': 'PERUS',
    'Pinheiros': 'PINHEIROS',
    'Pirituba': 'PIRITUBA',
    'Santana/Tucuruvi': 'SANTANA-TUCURUVI',
    'Santo Amaro': 'SANTO AMARO',
    'Sapopemba': 'SAPOPEMBA',
    'São Mateus': 'SAO MATEUS',
    'São Miguel': 'SAO MIGUEL',
    'Sé': 'SE',
    'Vila Maria/Vila Guilherme': 'VILA MARIA-VILA GUILHERME',
    'Vila Mariana': 'VILA MARIANA',
    'Vila Prudente': 'VILA PRUDENTE'
}

In [50]:
df_mortalidade_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (
    df_mortalidade_ajustado.loc[:, 'Subprefeitura residência'].map(subpref_map)
)

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16,ARICANDUVA-FORMOSA-CARRAO
1,Butantã,Doencas infec intest,14,BUTANTA
2,Campo Limpo,Doencas infec intest,13,CAMPO LIMPO
3,Capela do Socorro,Doencas infec intest,22,CAPELA DO SOCORRO
4,Casa Verde/Cachoeirinha,Doencas infec intest,12,CASA VERDE-CACHOEIRINHA
...,...,...,...,...
2139,Sapopemba,Demais causas de morte,429,SAPOPEMBA
2140,Sé,Demais causas de morte,733,SE
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523,VILA MARIA-VILA GUILHERME
2142,Vila Mariana,Demais causas de morte,682,VILA MARIANA


Para classificar as doenças em evitáveis e/ou tratáveis, foi elabora uma planilha complementar com os pesos de cada categoria para as causas específicas disponíveis no tabnet.

Vamos carregar essa planilha e utilizar como fator de ponderação.

In [51]:
data_path = path.join('dados', 'saude')
mortalidade_filename = 'DOENÇAS EVITAVEIS E-OU TRATAVEIS.xlsx'
mortalidade_file_path = path.join(data_path, mortalidade_filename)

df_evitaveis_trataveis = pd.read_excel(mortalidade_file_path, sheet_name=1)

df_evitaveis_trataveis

,Categorias específicas,Evitáveis,Tratáveis
0,Acid trânsito e transporte terrestres,1.0,NaN
1,Aids,1.0,NaN
2,Alcoolismo,1.0,NaN
3,Anemias,1.0,NaN
4,Aneurisma e dissecção aorta,0.5,0.5
...,...,...,...
62,Quedas acidentais,1.0,NaN
63,Septicemia,NaN,1.0
64,Suicídios,0.5,0.0
65,Tuberculose,0.5,0.5


In [52]:
df_evitaveis_trataveis = df_evitaveis_trataveis.fillna(0)
df_evitaveis_trataveis

,Categorias específicas,Evitáveis,Tratáveis
0,Acid trânsito e transporte terrestres,1.0,0.0
1,Aids,1.0,0.0
2,Alcoolismo,1.0,0.0
3,Anemias,1.0,0.0
4,Aneurisma e dissecção aorta,0.5,0.5
...,...,...,...
62,Quedas acidentais,1.0,0.0
63,Septicemia,0.0,1.0
64,Suicídios,0.5,0.0
65,Tuberculose,0.5,0.5


In [53]:
df_mortalidade_ajustado = df_mortalidade_ajustado.merge(df_evitaveis_trataveis,
                              left_on='Causa específica',
                              right_on='Categorias específicas',
                              how='left')

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Categorias específicas,Evitáveis,Tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16,ARICANDUVA-FORMOSA-CARRAO,Doencas infec intest,1.0,0.0
1,Butantã,Doencas infec intest,14,BUTANTA,Doencas infec intest,1.0,0.0
2,Campo Limpo,Doencas infec intest,13,CAMPO LIMPO,Doencas infec intest,1.0,0.0
3,Capela do Socorro,Doencas infec intest,22,CAPELA DO SOCORRO,Doencas infec intest,1.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,12,CASA VERDE-CACHOEIRINHA,Doencas infec intest,1.0,0.0
...,...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,429,SAPOPEMBA,Demais causas de morte,0.0,0.0
2140,Sé,Demais causas de morte,733,SE,Demais causas de morte,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523,VILA MARIA-VILA GUILHERME,Demais causas de morte,0.0,0.0
2142,Vila Mariana,Demais causas de morte,682,VILA MARIANA,Demais causas de morte,0.0,0.0


Nessa versão, a planilha não classica a causa específica `Infec menigocóccica`. Por ora, vou atribuir o peso 0 nas duas categorias, mas solicitarei a revisão dessa causa específica aos especialistas do GT Saúde.

In [54]:
df_mortalidade_ajustado.loc[:, 'Evitáveis'] = df_mortalidade_ajustado.loc[:, 'Evitáveis'].fillna(0)
df_mortalidade_ajustado.loc[:, 'Tratáveis'] = df_mortalidade_ajustado.loc[:, 'Tratáveis'].fillna(0)
df_mortalidade_ajustado = df_mortalidade_ajustado\
                            .drop(columns='Categorias específicas')

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Evitáveis,Tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16,ARICANDUVA-FORMOSA-CARRAO,1.0,0.0
1,Butantã,Doencas infec intest,14,BUTANTA,1.0,0.0
2,Campo Limpo,Doencas infec intest,13,CAMPO LIMPO,1.0,0.0
3,Capela do Socorro,Doencas infec intest,22,CAPELA DO SOCORRO,1.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,12,CASA VERDE-CACHOEIRINHA,1.0,0.0
...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,429,SAPOPEMBA,0.0,0.0
2140,Sé,Demais causas de morte,733,SE,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523,VILA MARIA-VILA GUILHERME,0.0,0.0
2142,Vila Mariana,Demais causas de morte,682,VILA MARIANA,0.0,0.0


In [55]:
df_mortalidade_ajustado = df_mortalidade_ajustado.rename(
    columns={'Evitáveis': 'Qtd evitáveis', 'Tratáveis': 'Qtd tratáveis'}
)

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16,ARICANDUVA-FORMOSA-CARRAO,1.0,0.0
1,Butantã,Doencas infec intest,14,BUTANTA,1.0,0.0
2,Campo Limpo,Doencas infec intest,13,CAMPO LIMPO,1.0,0.0
3,Capela do Socorro,Doencas infec intest,22,CAPELA DO SOCORRO,1.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,12,CASA VERDE-CACHOEIRINHA,1.0,0.0
...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,429,SAPOPEMBA,0.0,0.0
2140,Sé,Demais causas de morte,733,SE,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523,VILA MARIA-VILA GUILHERME,0.0,0.0
2142,Vila Mariana,Demais causas de morte,682,VILA MARIANA,0.0,0.0


In [56]:
df_mortalidade_ajustado.loc[:, 'Qtd evitáveis'] = (
    df_mortalidade_ajustado.loc[:, 'Qtd evitáveis']*df_mortalidade_ajustado.loc[:, 'Qtd óbitos']
)

df_mortalidade_ajustado.loc[:, 'Qtd tratáveis'] = (
    df_mortalidade_ajustado.loc[:, 'Qtd tratáveis']*df_mortalidade_ajustado.loc[:, 'Qtd óbitos']
)

C:\Users\855395\AppData\Local\Temp\ipykernel_12928\2988763396.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[16.0 14.0 13.0 ... 0.0 0.0 0.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mortalidade_ajustado.loc[:, 'Qtd evitáveis'] = (
C:\Users\855395\AppData\Local\Temp\ipykernel_12928\2988763396.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.0 0.0 0.0 ... 0.0 0.0 0.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mortalidade_ajustado.loc[:, 'Qtd tratáveis'] = (


In [57]:
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,16,ARICANDUVA-FORMOSA-CARRAO,16.0,0.0
1,Butantã,Doencas infec intest,14,BUTANTA,14.0,0.0
2,Campo Limpo,Doencas infec intest,13,CAMPO LIMPO,13.0,0.0
3,Capela do Socorro,Doencas infec intest,22,CAPELA DO SOCORRO,22.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,12,CASA VERDE-CACHOEIRINHA,12.0,0.0
...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,429,SAPOPEMBA,0.0,0.0
2140,Sé,Demais causas de morte,733,SE,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,523,VILA MARIA-VILA GUILHERME,0.0,0.0
2142,Vila Mariana,Demais causas de morte,682,VILA MARIANA,0.0,0.0


In [58]:
df_mortalidade_final = (
    df_mortalidade_ajustado
    .drop(columns=['Causa específica', 'Qtd óbitos'])
    .groupby(['Subprefeitura residência', 'SUBPREFEITURA_QLIK'])
    .sum()
    .reset_index()
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,1508.5,1064.0
1,Butantã,BUTANTA,1581.0,1104.5
2,Campo Limpo,CAMPO LIMPO,2189.0,1316.5
3,Capela do Socorro,CAPELA DO SOCORRO,2299.5,1479.0
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,1576.0,964.0
5,Cidade Ademar,CIDADE ADEMAR,1630.5,994.0
6,Cidade Tiradentes,CIDADE TIRADENTES,855.5,505.5
7,Ermelino Matarazzo,ERMELINO MATARAZZO,981.0,674.0
8,Freguesia/Brasilândia,FREGUESIA-BRASILANDIA,1933.0,1188.0
9,Guaianases,GUAIANASES,1108.0,680.5


In [59]:
df_mortalidade_final['Qtd evitáveis'] = (
    df_mortalidade_final['Qtd evitáveis']
    .astype(int)
)
df_mortalidade_final['Qtd tratáveis'] = (
    df_mortalidade_final['Qtd tratáveis']
    .astype(int)
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,1508,1064
1,Butantã,BUTANTA,1581,1104
2,Campo Limpo,CAMPO LIMPO,2189,1316
3,Capela do Socorro,CAPELA DO SOCORRO,2299,1479
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,1576,964
5,Cidade Ademar,CIDADE ADEMAR,1630,994
6,Cidade Tiradentes,CIDADE TIRADENTES,855,505
7,Ermelino Matarazzo,ERMELINO MATARAZZO,981,674
8,Freguesia/Brasilândia,FREGUESIA-BRASILANDIA,1933,1188
9,Guaianases,GUAIANASES,1108,680


In [60]:
df_mortalidade_final = (
    df_mortalidade_final
    .melt(
        id_vars=['Subprefeitura residência', 'SUBPREFEITURA_QLIK'],
        var_name='Evitáveis ou tratáveis',
        value_vars=['Qtd evitáveis', 'Qtd tratáveis'],
        value_name='Qtd óbitos'
    )
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Evitáveis ou tratáveis,Qtd óbitos
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,Qtd evitáveis,1508
1,Butantã,BUTANTA,Qtd evitáveis,1581
2,Campo Limpo,CAMPO LIMPO,Qtd evitáveis,2189
3,Capela do Socorro,CAPELA DO SOCORRO,Qtd evitáveis,2299
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,Qtd evitáveis,1576
...,...,...,...,...
59,São Miguel,SAO MIGUEL,Qtd tratáveis,1072
60,Sé,SE,Qtd tratáveis,1243
61,Vila Maria/Vila Guilherme,VILA MARIA-VILA GUILHERME,Qtd tratáveis,1003
62,Vila Mariana,VILA MARIANA,Qtd tratáveis,1127


In [61]:
df_mortalidade_final.loc[:, 'Evitáveis ou tratáveis'] = (
    df_mortalidade_final
    .loc[:, 'Evitáveis ou tratáveis']
    .str.removeprefix('Qtd ')
    .str.capitalize()
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Evitáveis ou tratáveis,Qtd óbitos
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,Evitáveis,1508
1,Butantã,BUTANTA,Evitáveis,1581
2,Campo Limpo,CAMPO LIMPO,Evitáveis,2189
3,Capela do Socorro,CAPELA DO SOCORRO,Evitáveis,2299
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,Evitáveis,1576
...,...,...,...,...
59,São Miguel,SAO MIGUEL,Tratáveis,1072
60,Sé,SE,Tratáveis,1243
61,Vila Maria/Vila Guilherme,VILA MARIA-VILA GUILHERME,Tratáveis,1003
62,Vila Mariana,VILA MARIANA,Tratáveis,1127


## Subprefeitura x Nível de atenção

Para que seja possível relacionar todos os indicadores sem nenhuma conversão nas medidas, precisamos de uma tabela com as dimensões em comum entre todas as tabelas para que seja feita a associação no Qlik. Por isso, vamos criar um dataframe que consistirá no produto cartesiano entre subprefeituras e nível de atenção padronizados.

In [62]:
df_niveis = pd.DataFrame([NIVEL_AB_VIGILANCIA, NIVEL_ESP, NIVEL_OUTROS],
                         columns=[NIVEL_COL_NAME])

df_subp_nivel = (
    df_mortalidade_final[['SUBPREFEITURA_QLIK']]
    .drop_duplicates()
    .merge(df_niveis, how='cross')
)

df_subp_nivel

,SUBPREFEITURA_QLIK,Nível de atenção padronizado
0,ARICANDUVA-FORMOSA-CARRAO,Atenção Básica e Vigilância
1,ARICANDUVA-FORMOSA-CARRAO,Atenção Especializada e Suporte profilático te...
2,ARICANDUVA-FORMOSA-CARRAO,Outros níveis de atenção
3,BUTANTA,Atenção Básica e Vigilância
4,BUTANTA,Atenção Especializada e Suporte profilático te...
...,...,...
91,VILA MARIANA,Atenção Especializada e Suporte profilático te...
92,VILA MARIANA,Outros níveis de atenção
93,VILA PRUDENTE,Atenção Básica e Vigilância
94,VILA PRUDENTE,Atenção Especializada e Suporte profilático te...


# Armazenamento dos dados

Finalmente, salvamos os arquivos como csv para utilizarmos no Qlik Sense.

In [63]:
base_path = path.join('data_output', 'saude')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [('consultas', df_consultas_final),
                 ('leitos-internacao-complementares', df_leitos_final),
                 ('despesas', df_despesa_agrupado),
                 ('mortalidade', df_mortalidade_final),
                 ('subprefeitura-nivel', df_subp_nivel)]:
    
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath, index=False)